In [100]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
from pymongo import MongoClient
import hdfs
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
import json
from datetime import datetime as dt
import datetime
import pandas as pd
import pytz

conn = MongoClient('mongo', 27017)
db = conn.tweets
collection = db.tweet

In [101]:
hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]
pprint([data_files, f'nombres de fichiers: {len(data_files)}', f'nombre de tweets: {len(data_files)*50}'], compact=True)

[['data-0.json', 'data-1.json', 'data-10.json', 'data-100.json',
  'data-101.json', 'data-102.json', 'data-103.json', 'data-104.json',
  'data-105.json', 'data-106.json', 'data-107.json', 'data-108.json',
  'data-109.json', 'data-11.json', 'data-110.json', 'data-111.json',
  'data-112.json', 'data-113.json', 'data-114.json', 'data-115.json',
  'data-116.json', 'data-117.json', 'data-118.json', 'data-119.json',
  'data-12.json', 'data-13.json', 'data-14.json', 'data-15.json',
  'data-16.json', 'data-17.json', 'data-18.json', 'data-19.json', 'data-2.json',
  'data-20.json', 'data-21.json', 'data-22.json', 'data-23.json',
  'data-24.json', 'data-25.json', 'data-26.json', 'data-27.json',
  'data-28.json', 'data-29.json', 'data-3.json', 'data-30.json', 'data-31.json',
  'data-32.json', 'data-33.json', 'data-34.json', 'data-35.json',
  'data-36.json', 'data-37.json', 'data-38.json', 'data-39.json', 'data-4.json',
  'data-40.json', 'data-41.json', 'data-42.json', 'data-43.json',
  'data-44.js

In [ ]:
collection.drop()

tweets = []

for i in data_files:
    df = spark.read.json(f'hdfs://hadoop-master:9000/user/kafka-data/{i}')
    json_data = df.toJSON().collect()
    for i in json_data:
        data = json.loads(i)
        hashtags = [i for i in data["hashtags"]]
        for h in hashtags:
            hours = 2
            hours_added = datetime.timedelta(hours = hours)
            new_datetime = dt.fromtimestamp(int(data["timestamp"])/1000)
            new_datetime = new_datetime + hours_added

            uniqued_hashtag = {"hashtag": h, "datetime": new_datetime}
            tweets.append(uniqued_hashtag)

collection.insert_many(tweets)
        
cursor = collection.find()
for record in cursor:
    print(record)